<a href="https://colab.research.google.com/github/jesperiksson/struc-mon/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import random
import numpy as np
import h5py
import scipy as sp
import scipy.signal

In [0]:
    use = 'LSTM'
    name = 'onlineCudnnData'
    #####################

    architecture = {
        'name' :use + name,
        'active_sensors' : ['90'],
        'predict' : 'accelerations', # accelerations or damage
        'path' : '/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/our_measurements3/e90',
        'random_mode' : 'test' # test or debug
        }
    sensor_dict = {}
    for i in range(len(architecture['active_sensors'])):
        sensor_dict.update({
            architecture['active_sensors'][i] : i
            })
    architecture.update({
        'sensors' : sensor_dict
        })
    plotting = {
        'prediction_performance' : True,
        'prediction_confusion_matrix' : True,
        'prediction_roc' : False, # To be implemented
        'forecast_performance' : True,
        'forecast_confusion_matrix' : True
        }

In [0]:
architecture.update({
            'model' : 'two_layer',
            # Net configuaration
            'n_units' : {'first' : 500, 'second' : 250},
            'bias' : True,
            'n_pattern_steps' : 1000, # Kan ändras
            'n_target_steps' : 100,
            'pattern_delta' : 40,
            # Sensor parameters
            'pattern_sensors' : ['90'],
            'target_sensor' : '90',
            'target_sensors' : ['90'],
            # Training parameters
            'batch_size' : 10,
            'data_split' : {'train':60, 'validation':20, 'test':20}, # sorting of data 
            'mode' : '1',
            'delta' : 1, # Kan ändras
            'Dense_activation' : 'tanh',
            'early_stopping' : True,
            'epochs' : 200,
            'learning_rate' : 0.001, # 0.001 by default
            'min_delta' : 0.01,
            'LSTM_activation' : 'tanh',
            'preprocess_type' : 'data',
            'patience' : 50,
            # Data interval
            'from' : 0,
            'to' : -1,
            # Model saving
            'save_periodically' : True,
            'save_interval' : 1, # Number of series to train on before saving
            # Classification
            'limit' : 0.9
        })

In [0]:
architecture

{'Dense_activation': 'tanh',
 'LSTM_activation': 'tanh',
 'active_sensors': ['90'],
 'batch_size': 10,
 'bias': True,
 'data_split': {'test': 20, 'train': 60, 'validation': 20},
 'delta': 1,
 'early_stopping': True,
 'epochs': 200,
 'from': 0,
 'learning_rate': 0.001,
 'limit': 0.9,
 'min_delta': 0.01,
 'mode': '1',
 'model': 'two_layer',
 'n_pattern_steps': 1000,
 'n_target_steps': 100,
 'n_units': {'first': 500, 'second': 250},
 'name': 'LSTMonlineCudnnData',
 'path': '/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/our_measurements3/e90',
 'patience': 50,
 'pattern_delta': 40,
 'pattern_sensors': ['90'],
 'predict': 'accelerations',
 'preprocess_type': 'data',
 'random_mode': 'test',
 'save_interval': 1,
 'save_periodically': True,
 'sensors': {'90': 0},
 'target_sensor': '90',
 'target_sensors': ['90'],
 'to': -1}

In [0]:
def fit_to_NN(
        data_split, 
        path, 
        healthy_percentage, 
        arch):
    
    types = arch['preprocess_type']
    paths = {}

    for i in range(len(arch['active_sensors'])):
        paths.update(
            {arch['active_sensors'][i] : path+'s'+arch['active_sensors'][i]+'/'})
    if arch['random_mode'] == 'debug':
        seed = 1
    elif arch['random_mode'] == 'test':
        seed = random.randint(0,10000)
    file_list = os.listdir(paths[arch['active_sensors'][0]])
    file_list.sort()
    random.Random(seed).shuffle(file_list)

    speeds = np.empty([len(file_list)])
    for i in range(len(file_list)):
        if len(file_list[i]) == 9:
            speeds[i] = int(file_list[i][0:5])
        elif len(file_list[i]) == 10:
            speeds[i] = int(file_list[i][0:6])
        else: 
            print('error')
    normalized_speeds = (speeds-min(speeds))/(max(speeds)-min(speeds))

    n_files = len(file_list)
    data_stack = {}
    preprocess_stack = {}
    peaks_stack = {}
    frequency_stack = {}
    extrema_stack = {}
    start = arch['from']
    to = arch['to']
    diff = to-start
    for i in range(n_files):
        data = [None]*len(paths)
        for j in range(len(paths)):
            mat = h5py.File(paths[arch['active_sensors'][j]] + file_list[i], 'r')
            data[j] = mat.get('acc')[1,start:to]

        if i/n_files < (data_split['train']/100):
            category = 'train'
        elif i/n_files > (data_split['validation']/100) and i/n_files < ((data_split['train']+data_split['validation'])/100):
            category = 'validation'
        else:
            category = 'test'
        if 'data' in types:
            data_stack.update({
                'batch'+str(i) : DataBatch(data,
                                 i,
                                 speeds[i]/1000,
                                 normalized_speeds[i],
                                 category,
                                 healthy_percentage)
                                })
        if 'frequency' in types:
            frequency_stack.update({
                'batch'+str(i) : frequencySpectrum(data,
                                 i,
                                 speeds[i]/1000,
                                 normalized_speeds[i],
                                 category,
                                 healthy_percentage)
                                })
        if 'peaks' in types:
            peaks_stack.update({
                'batch'+str(i) : peaks(data,
                                 i,
                                 speeds[i]/1000,
                                 normalized_speeds[i],
                                 category,
                                 healthy_percentage)
                                })
        if 'extrema' in types:
            extrema_stack.update({
            'batch'+str(i) : extrema(data,
                             i,
                             speeds[i]/1000,
                             normalized_speeds[i],
                             category,
                             healthy_percentage)
                            })

    preprocess_stack.update({
        'data' : data_stack,
        'frequency' : frequency_stack,
        'peaks' : peaks_stack,
        'extrema' : extrema_stack
        })    

    return preprocess_stack

In [0]:
class DataBatch():
    def __init__(self, data, batch_num, speed, normalized_speed, category, damage_state):
        self.data = np.array(data)
        self.unnormalized_data = np.array(data)
        for i in range(np.shape(data)[0]):
            self.data[i,:] = self.data[i,:]/max(abs(self.data[i,:]))
        #self.data = preprocessing.normalize(self.data)
        self.sensors = np.shape(data)[0]
        self.batch_num = batch_num
        self.category = category
        self.n_steps = np.shape(self.data)[1]
        self.speed = {'km/h' : speed, 'm/s' : (speed*3.6/10)}
        self.normalized_speed = normalized_speed
        self.damage_state = damage_state
        self.normalized_damage_state = damage_state/100
        self.timestep = 0.001
        self.timesteps = np.arange(0, self.n_steps, 1)
        self.steps = [None]*self.sensors
        self.indices = [None]*self.sensors
        self.delta = [None]*self.sensors
        for i in range(self.sensors):
            self.steps[i] = self.n_steps


In [0]:
def data_split_mode2(a):
    series_stack = {}
    damage_dir_list = os.listdir(a['path'])
    for j in range(len(damage_dir_list)):
        series_stack.update({
            damage_dir_list[j] : fit_to_NN(
                a['data_split'],
                a['path']+'/'+damage_dir_list[j]+'/', 
                int(damage_dir_list[j][:-1]),
                a)
            })
    return series_stack

def data_split_mode1(a):
    eval_series_stack = {}
    damage_dir_list = os.listdir(a['path'])
    for j in range(len(damage_dir_list)):
        if damage_dir_list[j] == '100%':
            eval_series_stack.update({
                damage_dir_list[j] : fit_to_NN(
                    a['data_split'],
                    a['path']+'/'+damage_dir_list[j]+'/',
                    int(damage_dir_list[j][:-1]),
                    a)
                })
        else:
            eval_series_stack.update({
                damage_dir_list[j] : fit_to_NN(
                    {'train' : 0, 'validation' : 0, 'test' : 100},
                    a['path']+'/'+damage_dir_list[j]+'/',
                    int(damage_dir_list[j][:-1]),
                    a)
                })
    return eval_series_stack

In [0]:
class peaks(DataBatch):
    def __init__(self, data, batch_num, speed, normalized_speed, category, damage_state):  
        super().__init__(data, batch_num, speed, normalized_speed, category, damage_state)
        self.peaks = [None]*self.sensors
        for i in range(self.sensors):
            self.indices[i], properties = sp.signal.find_peaks(
                self.data[i], 
                height = None, 
                threshold = None,
                distance = 2,
                prominence = None,
                width = None)
            self.peaks[i] = self.data[i][self.indices[i]]
            
            delta = np.diff(self.indices[i])
            self.delta[i] = delta/max(delta)
        self.n_steps = np.shape(self.peaks[0])[0] # overwrite data
        self.timesteps = self.indices[0]
        self.data = self.peaks 

In [0]:
    if architecture['mode'] == '1':
        series_stack = data_split_mode1(architecture)
        '''
        This is the normal case where all available data is divided into train/ test/ validation
        '''

    elif architecture['mode'] == '2':
        series_stack = data_split_mode2(architecture)
        '''
        This is special case where only healthy data is used for training and 
        all damaged data is used for testing.
        '''
    #series_stack

In [0]:
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential, Model, model_from_json
from tensorflow.python.keras.layers import Input, Dense, CuDNNLSTM, concatenate, Activation, Reshape, Bidirectional
from tensorflow.python.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.python.keras import metrics, regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras import backend
from tensorflow.python.keras.optimizers import RMSprop

In [0]:
class NeuralNet():
    def __init__(self,
                 arch,
                 name,
                 existing_model = False):

        self.arch = arch
        self.name = name
        self.target_sensor = self.arch['sensors'][self.arch['target_sensor']]
        self.pattern_sensors = self.arch['sensors'][self.arch['pattern_sensors'][0]]
        self.sensor_to_predict = arch['sensors'][arch['target_sensor']]
        if arch['early_stopping'] == True:
            self.early_stopping = [keras.callbacks.EarlyStopping(
                monitor='loss',
                min_delta=arch['min_delta'], 
                patience=arch['patience'],
                verbose=1,
                mode='auto',
                restore_best_weights=True)]

        else:
            self.early_stopping = None
        self.existing_model = existing_model
        self.n_sensors = len(arch['sensors'])    
        model_dict = {
            'single_layer' : set_up_model6(arch),
            'two_layer' : set_up_model7(arch)
            }     
        if self.existing_model == False:
            model = model_dict[arch['model']]

        elif self.existing_model == True:
            model_path = '/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/models/'+self.arch['name']+'.json'
            weights_path = '/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/models/'+self.arch['name']+'.h5'
            json_file = open(model_path)
            loaded_model_json = json_file.read()
            json_file.close()
            loaded_model = model_from_json(loaded_model_json)
            loaded_model.load_weights(weights_path)
            model = loaded_model
            print('\n Loaded model: ', name)
        else:
            raise Error
        optimizer = keras.optimizers.Adam(
            learning_rate = arch['learning_rate'],
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=False)
        model.compile(
            optimizer=optimizer, 
            loss=rmse, 
            metrics=[rmse])
        plot_model(model, to_file=name+'.png')
        model.summary()
        self.model = model
        self.score = None
        self.loss = [None]


In [0]:
    def train(self, series_stacks):
        tic = time.time()
        self.history = [None]
        self.loss = [None]
        self.val_loss = [None]
        keys = list(series_stacks.keys())
        for h in range(len(keys)):
            series_stack = series_stacks[keys[h]]
            print('\nTraining on ', keys[h],'% healthy data.\n')
            print('\n Number of series being used for training:', len(series_stack[self.arch['preprocess_type']]), '\n')
            for i in range(len(series_stack[self.arch['preprocess_type']])):
                series = series_stack[self.arch['preprocess_type']]['batch'+str(i)]
                if series.category == 'train' or series.category == 'validation':
                    print('\nFitting series: ', i, ' out of:', len(series_stack[self.arch['preprocess_type']]))
                    X, Y = generator(self, series)
                    patterns = {
                        'speed_input' : np.array([series.normalized_speed]),
                        'damage_input' : series.normalized_damage_state}
                    for j in range(len(self.arch['active_sensors'])):
                        patterns.update({
                            'accel_input_'+str(self.arch['pattern_sensors'][j]) : X # Ange vilken
                        })
                    targets = {
                        'accel_output_'+str(self.arch['pattern_sensors'][j]) : Y,
                        'damage_state_output' : series.normalized_damage_state
                        }           
                    history = self.model.fit(
                        x = X,#patterns,
                        y = Y,#targets, 
                        batch_size = self.arch['batch_size'],
                        epochs=self.arch['epochs'], 
                        verbose=1,
                        callbacks=self.early_stopping, #self.learning_rate_scheduler],
                        validation_split = self.arch['data_split']['validation']/100,
                        shuffle = True)
                    self.history.append(history)
                    self.loss.extend(history.history['loss'])
                    self.val_loss.extend(history.history['val_loss'])  
                    if self.arch['save_periodically'] == True and i % self.arch['save_interval'] == 0:
                        save_model(self.model,self.name)  
        self.model.summary()
        self.toc = np.round(time.time()-tic,1)
        print('Elapsed time: ', self.toc)
        return
    NeuralNet.train = train

In [0]:
def rmse(true, prediction):
    return backend.sqrt(backend.mean(backend.square(prediction - true), axis=-1))

In [0]:
def get_steps(self, series):
    steps = int(
        np.floor(
            (series.n_steps-(self.arch['n_pattern_steps']+self.arch['n_target_steps']))/self.arch['pattern_delta']
        )
    )
    return steps  

In [0]:
    def evaluation(self, series_stack):
        scores = []
        speeds = []
        damage_states = []
        for i in range(len(series_stack[self.arch['preprocess_type']])):
            series = series_stack[self.arch['preprocess_type']]['batch'+str(i)]
            if series.category == 'test':
                X, Y = generator(self, series)
                score = self.model.evaluate(
                    x = X,
                    y = Y,
                    batch_size = self.arch['batch_size'],
                    verbose = 1,
                    return_dict = True)
                speeds.extend([series.speed['km/h']])
                scores.extend([score['rmse']])
                damage_states.extend([series.damage_state])
            
        results = {
            'scores' : scores[:],
            'speeds' : speeds[:],
            'steps' : len(speeds[:]),
            'damage_state' : damage_states[:]
        }
        return results
    NeuralNet.evaluation = evaluation

In [0]:
def generator(self, batch):
    '''
    Generator for when to use Peak accelerations and locations
    Each series of data is so big it has to be broken down
    '''
    steps = get_steps(self, batch)
    #print(steps)
    X = np.empty([steps,self.arch['n_pattern_steps'], len(self.arch['pattern_sensors'])])
    Y = np.empty([steps,self.arch['n_target_steps']])
    for j in range(len(self.arch['pattern_sensors'])):     
        for k in range(steps):    
            pattern_start = k*self.arch['pattern_delta']
            pattern_finish = k*self.arch['pattern_delta']+self.arch['delta']*self.arch['n_pattern_steps']
            target_start = k*self.arch['pattern_delta']+self.arch['delta']*self.arch['n_pattern_steps'] # +1?
            target_finish = k*self.arch['pattern_delta']+self.arch['delta']*(self.arch['n_pattern_steps']+self.arch['n_target_steps'])
            X[k,:,j] = batch.data[j][pattern_start:pattern_finish]
            Y[k,:] = batch.data[j][target_start:target_finish]
    return X, Y

In [0]:
def save_model(model,name):
    model_json = model.to_json()
    with open('/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/models/'+name+'.json', 'w') as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
    model.save_weights('/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/models/'+name+'.h5')
    print('Saved model:', name)

In [0]:
def plot_loss(self, name):
    plt.figure()
    plt.plot(range(len(self.loss)), self.loss, 'bo', label='Training loss', linewidth=0.3)
    plt.plot(range(len(self.val_loss)), self.val_loss, 'ro', label='Validation loss', linewidth=0.3)
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss - RMSE')
    plt.legend()
    plt.savefig(fname = name+'_loss_plot.png')
    plt.show() 

In [0]:
def set_up_model6(arch): # Vanilla 

    accel_input = Input(
        shape=(
            arch['n_pattern_steps'], 
            1),
        name = 'accel_input_90')

    hidden_lstm_1 = CuDNNLSTM(
        arch['n_units']['first'],
        batch_input_shape = (
            arch['batch_size'],
            arch['n_pattern_steps'],
            1),
        #activation = 'tanh',
        #recurrent_activation = 'sigmoid',
        #use_bias = True,
        #dropout = 0.1,
        #recurrent_dropout = 0,
        #unroll = False,
        return_sequences = True,
        stateful = False)(accel_input)

    output = Dense(
        arch['n_target_steps'], 
        activation='tanh', 
        name='peak_output_90')(hidden_lstm_1)

    model = Model(inputs = accel_input, outputs = output)

    return model

In [0]:
def set_up_model7(arch):

    accel_input = Input(
        shape=(
            arch['n_pattern_steps'], 
            1),
        name = 'accel_input_90')

    hidden_lstm_1 = CuDNNLSTM(
        arch['n_units']['first'],
        batch_input_shape = (
            arch['batch_size'],
            arch['n_pattern_steps'],
            1),
        #activation = 'tanh',
        #recurrent_activation = 'sigmoid',
        #use_bias = True,
        #dropout = 0.1,
        #recurrent_dropout = 0,
        #unroll = False,
        return_sequences = True,
        stateful = False)(accel_input)

    hidden_lstm_2 = CuDNNLSTM(
        arch['n_units']['second'],
        batch_input_shape = (
            arch['batch_size'],
            arch['n_pattern_steps'],
            1),
        #activation = 'tanh',
        #recurrent_activation = 'sigmoid',
        #use_bias = True,
        #dropout = 0.1,
        #recurrent_dropout = 0,
        #unroll = False,
        stateful = False)(hidden_lstm_1)

    output = Dense(
        arch['n_target_steps'], 
        activation='tanh', 
        name='peak_output_90')(hidden_lstm_2)

    model = Model(inputs = accel_input, outputs = output)

    return model

In [61]:
    machine_stack = {}
    
    for i in range(len(architecture['target_sensors'])):
        architecture['target_sensor'] = architecture['target_sensors'][i]
        name = architecture['name']
        try:
            f = open('/content/drive/My Drive/KTH/Examensarbete/struc-mon (kopia)/models/'+name+'.json')
            machine_stack.update({
                name : NeuralNet(architecture,
                     name,
                     existing_model = True)
            })
        except IOError:    
            machine_stack.update({
                name : NeuralNet(architecture,
                     name,
                     existing_model = False)
            })
        NeuralNet.train(machine_stack[name], series_stack)  
        save_model(machine_stack[name].model, name)
        plot_loss(machine_stack[name], name)  
        
        score_stack = {}
        keys = list(series_stack)
        
        for j in range(len(keys)):
            score_stack.update({
                keys[j] : NeuralNet.evaluation(machine_stack[name], series_stack[keys[j]])
            })


 Loaded model:  LSTMonlineCudnnData
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
accel_input_90 (InputLayer)  [(None, 1000, 1)]         0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 1000, 500)         1006000   
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 250)               752000    
_________________________________________________________________
peak_output_90 (Dense)       (None, 100)               25100     
Total params: 1,783,100
Trainable params: 1,783,100
Non-trainable params: 0
_________________________________________________________________

Training on  57% % healthy data.


 Number of series being used for training: 6 


Training on  67% % healthy data.


 Number of series being used for training: 6 


Training on  90% % healthy data.


 Number 

In [0]:
    def forecast(machines, manual):
        # Machines
        machine_keys = list(machines.keys())
        # Shortcuts
        machine = machines[machine_keys[0]]
        delta = machine.arch['delta']
        n_pattern_steps = machine.arch['n_pattern_steps']
        n_target_steps = machine.arch['n_target_steps']
        # Series
        key = 'batch'+str(manual['series_to_predict']%len(manual['stack'][machine.arch['preprocess_type']]))
        n_steps = manual['stack'][machine.arch['preprocess_type']][key].n_steps
        series = manual['stack'][machine.arch['preprocess_type']][key]
        n_series = int((n_steps-n_pattern_steps)/n_target_steps)
        # Initial
        initial_indices = np.arange(0,delta*n_pattern_steps,delta)
        patterns = {}
        for i in range(len(machine.arch['pattern_sensors'])):
            patterns.update({ 
            'accel_input_'+machine.arch['pattern_sensors'][i] : 
                np.reshape(
                    series.data[machine.sensor_to_predict][initial_indices], 
                    [1,machine.arch['n_pattern_steps']]
                )
            })
        forecasts = patterns.copy()
        evaluation = {}
        for i in range(n_series+1):
            old_patterns = patterns.copy()
            for j in range(len(machine_keys)):
                machine = machines[machine_keys[j]] # Pick machine
                prediction = machine.model.predict(
                    old_patterns,
                    batch_size = 1, 
                    verbose=0,
                    steps = 1) # Make prediction with machine
                pattern = patterns['accel_input_'+machine.arch['pattern_sensors'][j]] # Extract pattern
                pattern = np.delete(pattern,np.s_[0:n_target_steps:delta],1) # Remove first entty
                pattern = np.hstack([pattern,prediction]) # Add prediciton last
                patterns.update({
                    'accel_input_'+machine.arch['pattern_sensors'][j] : pattern
                    }) # Update patterns dict
                forecast = forecasts['accel_input_'+machine.arch['pattern_sensors'][j]] #Extract forecast
                if i == n_series: # Edge case for last bit                   
                    forecast = np.hstack(
                        [forecast,
                        prediction[:,:n_steps%(n_series*n_target_steps+n_pattern_steps)]]
                        ) # Update forecast
                else:
                    forecast = np.hstack([forecast,prediction]) # Update forecast
                forecasts.update({
                    'accel_input_'+machine.arch['pattern_sensors'][j] : forecast
                    }) # Update forecasts dict
        score = rmse_np(
            series.data[j][n_pattern_steps:], 
            forecasts['accel_input_'+machine.arch['pattern_sensors'][j]][0][n_pattern_steps:])
        speed = series.speed['km/h']
        damage_state = series.damage_state
        return forecasts, (score, speed, damage_state)
NeuralNet.forecast = forecast

In [0]:
    for i in range(len(architecture['target_sensors'])):
        architecture['target_sensor'] = architecture['target_sensors'][i]
        name = architecture['name']
        try:
            f = open('models/'+name+'.json')
            machine_stack.update({
                name : NeuralNet(architecture,
                     name,
                     existing_model = True)
            })
        except IOError:    
            machine_stack.update({
                name : NeuralNet(architecture,
                     name,
                     existing_model = False)
            })
            NeuralNet.train(machine_stack[name], series_stack)  
            save_model(machine_stack[name].model, name)
            plot_loss(machine_stack[name], name)

In [0]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
def plot_performance(score_stack, a, pod): # pod = prediction or forecast
    cmap = plt.cm.rainbow
    norm = colors.Normalize(vmin=33,vmax=100)
    percentage_keys = list(score_stack)
    for i in range(len(percentage_keys)): # Iterates over percentages
        for j in range(len(score_stack[percentage_keys[i]]['speeds'])):
            plt.plot(score_stack[percentage_keys[i]]['speeds'][j], 
                     score_stack[percentage_keys[i]]['scores'][j], 
                     color=cmap(norm(score_stack[percentage_keys[i]]['damage_state'][j])), 
                     marker='o')
    plt.xlabel('Speed [km/h]')
    plt.ylabel('Root Mean Square Error')
    plt.title('Sample scores for '+pod+' at sensor ' + str(a['target_sensor']))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm)
    cbar.set_label('Young\'s modulus percentage', rotation=270)
    plt.legend()
    plt.savefig(fname = a['name']+pod+'_performance_plot.png')
    plt.show()

In [0]:
plot_performance(score_stack, architecture, 'prediction')

In [0]:
def rmse_np(true, prediction):
    return np.sqrt(np.mean(np.square(prediction - true), axis=-1))


In [0]:
def plot_forecast(forecast, manual, a):
    key = 'batch'+str(manual['series_to_predict'])
    forecast_keys = list(forecast.keys())
    for i in range(len(forecast_keys)): 
        plt.subplot(len(forecast_keys),1,i+1)
        plt.plot(
            manual['stack'][a['preprocess_type']][key].timesteps, 
            forecast[forecast_keys[i]][0], 
            'b', 
            linewidth=0.4)
        plt.plot(
            manual['stack'][a['preprocess_type']][key].timesteps, 
            manual['stack'][a['preprocess_type']][key].data[i], 
            'r', 
            linewidth=0.4)
        plt.xlabel('timesteps')
        plt.ylabel('accelerations')
        #plt.title('Forecast for response at '+str(manual['stack'][a['preprocess_type']][key].speed['km/h']+' km/h'))
        plt.legend(['Forecast', 'Signals']) 
    plt.savefig(fname = a['name']+'series'+str(manual['series_to_predict'])+'_forecast_plot.png')
    plt.show() 
plot_forecast(forecast, prediction_manual, architecture)

In [0]:
    prediction_score = {}
    for i in range(len(keys)):
        #print(series_stack[keys[i]])
        scores = []
        speeds = []
        damage_states = []
        for j in range(len(series_stack[keys[i]][architecture['preprocess_type']])):
            if series_stack[keys[i]][architecture['preprocess_type']]['batch'+str(j)].category == 'test':
                prediction_manual = {
                    'series_to_predict' : j,
                    'stack' : series_stack[keys[i]]
                }
                #prediction = NeuralNet.prediction(machine_stack[name], prediction_manual)
                #plot_prediction(prediction, prediction_manual, use)
                forecast, tup = NeuralNet.forecast(machine_stack, prediction_manual)
                scores.extend([tup[0]])
                speeds.extend([tup[1]])
                damage_states.extend([tup[2]])
            else:
                continue
        prediction_score.update({
            keys[i] : {'scores' : scores, 'speeds' : speeds, 'damage_state' : damage_states}           
            })
        